In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

In [4]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas.columns=['idaviso','idpostulante']

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)



In [5]:
#Genero no-postulaciones
from IPython.display import clear_output

postulaciones3=postulaciones;
postulaciones3['sepostulo']=0;
serie=postulantes['idpostulante'];
i=0
while (len(postulaciones3)>len(serie)):
    global i
    serie=serie.sample(frac=1).reset_index(drop=True)
    serie=pd.concat([serie.sample(frac=1),serie], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()
serie=serie.reset_index(drop=True)
postulaciones3['idpostulante']=serie.iloc[0:len(postulaciones3)]
postulaciones3.to_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/1.csv",index=False)
postulaciones3.head()

0
1
2
3


,idaviso,idpostulante,sepostulo
0,1112257047,NzeV9b5,0
1,1111920714,1QkYZPj,0
2,1112346945,8M2dWpx,0
3,1112345547,6OMWoO,0
4,1112237522,0zr8mWM,0


In [7]:
#Genero no-postulaciones teniendo en cuenta vistas
from IPython.display import clear_output

postulaciones3=postulaciones;
postulaciones3=postulaciones3.iloc[0:len(postulaciones)//4]
serie=postulantes['idpostulante'];
i=0
while (len(postulaciones3)>len(serie)):
    global i
    serie=serie.sample(frac=1).reset_index(drop=True)
    serie=pd.concat([serie.sample(frac=1),serie], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()
serie=serie.reset_index(drop=True)


print("hola")
postulaciones3['idpostulante']=serie.iloc[0:len(postulaciones3)]

postulaciones3=pd.concat([postulaciones3,vistas.sample((len(postulaciones)*3)//2)],ignore_index=False)
postulaciones3['sepostulo']=0;
postulaciones3.drop_duplicates()
postulaciones3.to_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/2.csv",index=False)


0
1
hola


C:\Users\Pocho\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Pocho\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [6]:
#Genero no-postulaciones teniendo en cuenta vistas y anuncios
from IPython.display import clear_output

postulaciones4=postulaciones;
postulaciones4=postulaciones4.iloc[0:len(postulaciones)*3//4]
serie_post=postulantes['idpostulante']
serie_anun = avisos["idaviso"]
i=0
while (len(postulaciones4)>len(serie_post)):
    global i
    serie_post=serie_post.sample(frac=1).reset_index(drop=True)
    serie_post = pd.concat([serie_post.sample(frac=1),serie_post], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()

while ( len(postulaciones4)>len(serie_anun) ):
    global i
    serie_anun=serie_anun.sample(frac=1).reset_index(drop=True)
    serie_anun = pd.concat([serie_anun.sample(frac=1),serie_anun], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()
        
serie_post=serie_post.reset_index(drop=True)
serie_anun = serie_anun.reset_index(drop=True)

postulaciones4['idpostulante']= serie_post.iloc[:len(postulaciones4)]
postulaciones4["idaviso"] = serie_anun.iloc[:len(postulaciones4)]
print(postulaciones4.head(10))
postulaciones4=pd.concat([postulaciones4,vistas.sample((len(postulaciones)*3)//2)],ignore_index=False)
postulaciones4['sepostulo']=0;
postulaciones4.drop_duplicates()
postulaciones4.to_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/3.csv",index=False)


10
11


C:\Users\Pocho\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Pocho\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


      idaviso idpostulante  sepostulo
0  1112338512      NzrZ6pM          1
1  1112262617      zvPkR5b          1
2  1112307057       YZPrvJ          1
3  1112351911       EkDl8b          1
4  1112509963      6rQkj2L          1
5  1112442048      pzjkR0b          1
6  1112337685      DrpbjjZ          1
7  1112233648      0zkj2rq          1
8  1112354346      rmR1Z3A          1
9  1111472660      OqreeXN          1


C:\Users\Pocho\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [5]:
#AZAR TOTAL - USAR ESTO

from IPython.display import clear_output

postulaciones3=postulaciones;
postulaciones3['sepostulo']=0;

serie=avisos['idaviso'];
i=0
while (len(postulaciones3)>len(serie)):
    global i
    serie=serie.sample(frac=1).reset_index(drop=True)
    serie=pd.concat([serie.sample(frac=1),serie], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()
serie=serie.reset_index(drop=True)



seriep=postulantes['idpostulante'];
i=0
while (len(postulaciones3)>len(seriep)):
    global i
    seriep=seriep.sample(frac=1).reset_index(drop=True)
    seriep=pd.concat([seriep.sample(frac=1),seriep], ignore_index=False)
    print(i)
    i+=1
    if(i%10==0): clear_output()
seriep=seriep.reset_index(drop=True)


postulaciones3['idaviso']=serie.iloc[0:len(postulaciones3)]
postulaciones3['idpostulante']=seriep.iloc[0:len(postulaciones3)]
postulaciones3.to_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv",index=False)

0
1
2
3
4
5
6
7
8
0
1
2
3
